#https://www.hackerrank.com/challenges/cut-the-tree

In [ ]:
# solution 1
class Node:
​
    def __init__(self, value):
        self.value = value
        self.total = 0
        self.visited = False
        self.adj_list = []
​
N = input()
​
values = map(int, raw_input().split())
# nodes start with index 1
nodes = [ None ]
nodes.extend([ Node(val) for val in values ])
​
for i in xrange(1, N):
    v1, v2 = map(int, raw_input().split())
    nodes[v1].adj_list.append(nodes[v2])
    nodes[v2].adj_list.append(nodes[v1])
​
# pick node #1 (randomly) as root and make the graph a tree
root = nodes[1]
q = []
q.append(root)
index = 0
while index < len(q):
    cur_node = q[index]
    for child in cur_node.adj_list:
        child.adj_list.remove(cur_node)
        q.append(child)
    index += 1
​
# calculate totals
index = len(q) - 1
while index >= 0:
    cur_node = q[index]
    for child in cur_node.adj_list:
        cur_node.total += child.total
    cur_node.total += cur_node.value
    index -= 1
​
min_diff = 1 << 32
q = []
q.append(root)
while index < len(q):
    cur_node = q[index]
    for child in cur_node.adj_list:
        t1 = child.total
        t2 = root.total - t1
        # no need to traverse a subtree if diff >= min_diff
        if abs(t1 - t2) < min_diff:
            min_diff = abs(t1 - t2)
            q.append(child)
    index += 1
​
print min_diff
​
​
​
​
​
​
# solution 2
N = int(raw_input())
weights = map(int, raw_input().split())
edges = []
for i in range(N-1):
    edges.append(set(map(lambda x: int(x)-1, raw_input().split())))
    
class Graph(object):
    def __init__(self, weights, edges):
        self.N = len(weights)
        self.weights = weights
        self.T = sum(weights)
        self.edges = edges
        self.adj_list = {}
        for i in range(self.N):
            self.adj_list[i] = []
        for (v1, v2) in self.edges:
            self.adj_list[v1].append(v2)
            self.adj_list[v2].append(v1)
​
    def dfs(self):
        self.min_num = self.T
        self.explored = set()
        self.sums = {}
        self.stack = [0]
        while len(self.stack) > 0:
            self.explore(self.stack[-1])
    
    def explore(self, vert):
        if vert in self.explored:
            total = self.weights[vert]
            for v in self.adj_list[vert]:
                if v in self.sums:
                    total += self.sums[v]
            self.sums[vert] = total
            self.min_num = min(self.min_num, abs(self.T-2*total))
            self.stack.pop()
            return
        self.explored.add(vert)
        total = self.weights[vert]
        for v in self.adj_list[vert]:
            if v not in self.explored:
                self.stack.append(v)
        
g = Graph(weights, edges)
g.dfs()
print g.min_num
​
​
​
​
​
​
# solution 3
#total is the sum of the tree
​
#parent is an array of size n, initially set to -1
#  after execution, it indicates the parent of i
​
#subtree is an array of size n, initially set to vals[i]
#  after execution, subtree[i] is the weight of the subtree rooted at i
​
#minCut is an array of size n, initially set to total
#  after execution, minCut[i] is the smallest cut one can make in the subtree rooted at i
#  cut a child or a descendant
​
def bfsCut(n, adj, total, parent, subtree, minCut):
    
    queue = [0]
    parent[0] = 0
    
    #find bfs ordering
    for index in range(n):
        i = queue[index]
                
        for neigh in adj[i]:
            #ignore edge to discovered node (i's parent)
            if parent[neigh] > -1: continue
                
            queue.append(neigh)
            parent[neigh] = i
        
        pass
    
    #from leaf to root...
    while len(queue) > 1:
        i = queue.pop()
        
        #compute the cut cost of this subtree
        rem = total-subtree[i]
        cut = abs(rem-subtree[i])
        
        #propagate cut cost and subtree weight to parent
        minCut[parent[i]] = min(minCut[parent[i]], cut, minCut[i])        
        subtree[parent[i]] += subtree[i]
        pass
    
    return minCut[0]
    
if __name__ == "__main__":
​
    n = int(raw_input())
    vals = list(map(int, raw_input().split()))
    total = sum(vals)
​
    adj,parent,subtree,minCut = [],[],[],[]
    for i in range(n):
        adj.append([])
        parent.append(-1)
        subtree.append(vals[i])
        minCut.append(total)
    
    for i in range(n-1):
        (x,y) = map(int,raw_input().split())
        adj[x-1].append(y-1)
        adj[y-1].append(x-1)
        
    print bfsCut(n, adj, total, parent, subtree, minCut)
​
​
​
​
​
# solution 4
import sys
​
data = sys.stdin
​
#collect node count
N = int(data.readline())
​
#collect node values
values = [int(x) for x in data.readline().split()]
​
#pre-populate tree with node values, scores
tree = [[] for x in xrange(N+1)]
​
#collect edge arrays
​
edge_list = [[int(y) for y in x.split()] for x in data.readlines()]
​
#populate tree with edges
for i in xrange(N-1):
    edge = edge_list[i]
    tree[edge[0]].append(edge[1])
    tree[edge[1]].append(edge[0])
​
root = edge_list[0][0]
child_scores = [0 for i in xrange(N+1)]
results = []
checked_nodes = set()
​
​
node = root
while node:
    checked_nodes.add(node)
    unvisited = [x for x in tree[node] if x not in checked_nodes]
    if unvisited:
        destination = unvisited[0]
        tree[node].remove(destination)
        node = destination
    else:
        visited = [x for x in tree[node]]
        if visited:
            parent = visited[0]
            current_score = child_scores[node] + int(values[node-1])
            child_scores[parent] += current_score
            results.append(current_score)
            node = parent
        else:
            node = ''
        
#sort for number closest to the median of the total score, print as result
max_val = sum(values)
closest = min(results,key=lambda x:abs((max_val/2)-x))
print(abs((max_val - closest)-closest))
​
​
​
​
​
# solution 5
N = input()
arr = map(int, raw_input().split())
edges = [[] for n in range(N)]
for n in range(N-1):
  i,j = map(int,raw_input().split())
  i = i - 1
  j = j - 1
  edges[i].append(j)
  edges[j].append(i)
st = [0]
while st:
  i = st.pop()
  for j in edges[i]:
    edges[j].remove(i)
    st.append(j)
st = [0]
n = 0
while n<len(st):
  i = st[n]
  for j in edges[i]:
    st.append(j)
  n = n + 1
sums = [None for i in range(N)]
while (st):
  i = st.pop()
  sums[i] = arr[i] + sum([sums[j] for j in edges[i]])
minDiff = float('inf')
for i in range(N):
  for j in edges[i]:
    diff = abs(sums[0] - 2*sums[j])
    minDiff = min(minDiff, diff)
print minDiff
  
​
​
​
​
​
​
​
​
# solution 6
N = int(raw_input())
G = map(int,raw_input().split())
Gsum = reduce(lambda x,y:x+y,G)
​
T = [[] for x in range(N)]
for N_i in xrange(N-1):
    i,j = map(int,raw_input().split())
    T[i-1].append(j)
    T[j-1].append(i)
# print T
​
Tree_diff = Gsum
​
A = [0 for x in range(N)]
def traverse():
  global Tree_diff
  i = 1
  while len(T[i-1]) > 0:
    # print "visit",i,T
    v = T[i-1].pop()
    if v < 0:
      v = -v
      A[i-1] += G[i-1]
      A[v-1] += A[i-1]
      Tree_diff = min(Tree_diff,abs((Gsum-A[i-1])-A[i-1]))
      i = v
    else:
      # print "remove",T[v-1],i
      T[v-1].remove(i)
      T[v-1].insert(0,-i)
      i = v
​
traverse()
# print A
print Tree_diff
​
​
​
​
​
​
​
# solution 7
def DFS(node,edges,visited,nodeWeight,sumWeight):
    lst = [node]
    children = {}
    children[node] = [v for v in edges[node]]
    visited[node] = True
    sumWeight[node] = nodeWeight[node] 
    while(len(lst) > 0):
        cur = lst[len(lst)-1]
        if len(children[cur]) > 0:
            child = children[cur].pop()
            if not visited[child]:
                visited[child] = True
                sumWeight[child] = nodeWeight[child]
                lst.append(child)
                children[child] = [v for v in edges[child]]
        else:
            last = lst.pop()
            if(len(lst) > 0):
                sumWeight[lst[len(lst)-1]] += sumWeight[last]
​
    
​
N = input()
edges = dict((k+1,[]) for k in range(N))
w = [int(s) for s in raw_input().split(" ")]
nodeWeight = dict((i+1,w[i]) for i in range(N))
sumWeight = dict((i+1,0) for i in range(N))
for i in range(1,N):
    (u,v) = (int(k) for k in raw_input().split(" "))
    edges[u].append(v)
    edges[v].append(u)
visited = dict((k,False) for k in range(1,N+1))
DFS(1,edges,visited,nodeWeight,sumWeight)
total = sumWeight[1]
m = total
for u in range(2,N+1):
    m = min(abs(total-2*sumWeight[u]),m)
print m
​
​
​
​
# solution 8
import sys, resource, os
sys.setrecursionlimit(100005)
​
resource.setrlimit(resource.RLIMIT_STACK, (65532000, 65532000))
​
​
def dfs(nd, pre):
    sum_[nd-1] = arr[nd-1]
    
    for it in tree[nd]:
        if it != pre:
            sum_[nd-1] += dfs(it, nd)
    
    return sum_[nd-1]
​
def dfs2(nd, pre):
    global answer
    for it in tree[nd]:
        if it != pre:
            answer=min(answer, abs(sum_[0]-2*sum_[it-1]))
            dfs2(it, nd)
​
answer = 10**9
​
N = int(raw_input().strip())
arr = map(int, raw_input().strip().split())
​
sum_ = [0]*N
tree = {}
binary = []
for i in xrange(N-1):
    T1 , T2 = map(int, raw_input().strip().split())
    binary.append((T1, T2))
    if T1 in tree:
        if T2 not in tree[T1]:
            tree[T1].append(T2)
    else:
        tree[T1] = [T2]
​
    if T2 in tree:
        if T1 not in tree[T2]:
            tree[T2].append(T1)
    else:
        tree[T2] = [T1]
​
dfs(1, 0)
dfs2(1, 0)
print answer
​
​
​
​
​
​
​
# solution 9
count = int(raw_input().strip())
values = map(int, raw_input().strip().split(' '))
neighbors = [list() for _ in xrange(len(values))]
for _ in xrange(len(values) - 1):
    n1, n2 = map(int, raw_input().strip().split(' '))
    neighbors[n1 - 1].append(n2 - 1)
    neighbors[n2 - 1].append(n1 - 1)
​
total_sum = sum(values)
best_diff = 9999999999999999999999
nodes = filter(lambda n: len(neighbors[n]) == 1, range(len(values)))
while len(nodes) > 1:
    index = nodes.pop()
    neighbor = neighbors[index][0]
    if len(neighbors[neighbor]) == 2:
        nodes.append(neighbor)
    best_diff = min(best_diff, abs(total_sum - 2* values[index]))
    values[neighbor] += values[index]
    neighbors[neighbor].remove(index)
print best_diff